# **Retrieval-Augmented Generation (RAG) Model for QA Bot**
## **Introduction**
This project involves building a Retrieval-Augmented Generation (RAG) model using Pinecone and Hugging Face Transformers. The model retrieves relevant chunks of text from a document and uses a question-answering pipeline to provide accurate answers. Since the **OpenAI API expired, the Hugging Face Transformers model is used** for embeddings and question-answering
## **Requirements**
Before running the code, install the necessary libraries. Run the following commands in a code cell:


In [ ]:
!pip install sentence-transformers
!pip install pinecone-client
!pip install transformers
!pip install rapidocr-onnxruntime
!pip install langchain
!pip install -U langchain-community
!pip install pypdf

# **Importing the libraries**

In [ ]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Pinecone
import pinecone
import os
from pinecone import ServerlessSpec
from transformers import pipeline

# **Initializing Pinecone Database**
Please change the index_name, If you're trying to run the program more than once since the index has already been created in the name mentioned below when you clicked run for the first time.

In [ ]:
# Initializing Pinecone
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', '9e390448-86d4-4123-b97b-bf77249a65ab')
pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment='us-east-1')

# Defining the index name and dimension
index_name = "test2"
dimension = 384

# Defining serverless index configuration
index_config = ServerlessSpec(
    cloud="aws",
    region="us-east-1"
)

# Checking if the index already exists
# Using pinecone_client to interact with Pinecone
if index_name not in pinecone_client.list_indexes():
    pinecone_client.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=index_config
    )


# **Data Preprocessing**

In [ ]:
!pip install pypdf
loader = PyPDFLoader("https://arxiv.org/pdf/2408.00716v1", extract_images=True)
data = loader.load()

# Printing document details
print(f'You have {len(data)} document(s) in your data')
print(f'There are {len(data[0].page_content)} characters in your sample document')
print(f'Here is a sample: {data[0].page_content[:200]}')

# Spliting document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(data)
print(f'Now you have {len(texts)} documents')

# **Model Building**

In [ ]:
# Loading the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to get embeddings
def get_embeddings(texts):
    embeddings = model.encode(texts)
    return embeddings

# **Preparing Data for Pinecone**

In [ ]:
import pinecone

# Preparing data for Pinecone
chunk_texts = [doc.page_content for doc in texts]
chunk_ids = [f"id_{i}" for i in range(len(chunk_texts))]

# Getting the embeddings
chunk_embeddings = get_embeddings(chunk_texts)

# Connecting to Pinecone and updating the data
index = pinecone_client.Index(index_name)
index.upsert(vectors=list(zip(chunk_ids, chunk_embeddings.tolist())))
print("Embeddings successfully inserted into Pinecone.")


# **Initializing Hugging Face Transformer QA Pipeline**
Since my OpenAI API key expired, I have replaced it with the Hugging face Transformer

In [ ]:
# Initializing the QA pipeline
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Function to answer a question using the document chunks
def answer_question(question):
    # Using the Pinecone index to search for relevant chunks
    query_embedding = get_embeddings([question])[0]
    query_embedding_list = query_embedding.tolist()

    # Useing keyword arguments for index.query()
    results = index.query(vector=query_embedding_list, top_k=5)  # Retrieving top 5 results
    relevant_chunks = [result['id'] for result in results['matches']]

    # Combining the content of the relevant chunks
    relevant_texts = [chunk_texts[int(chunk_id.split('_')[1])] for chunk_id in relevant_chunks]
    context = " ".join(relevant_texts)

    # Printing the answer from the Hugging Face model
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# **QA Bot**
The document is based on the usage of the NLP Framework for Hotel Recommendation based on
user’s text reviews.
Ask Questions like the following:
1. What is AI?
2. What is ML?
3. What is RNN?
4. What is NLP used for?

In [ ]:
# Example question
question = input("Enter the Question")
answer = answer_question(question)
print("Answer:", answer)

Enter the Questionwhat is AI
Answer: artificial 
intelligence
